In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql.functions import *
from pyspark.sql.types import *
#sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 
#sc.install_pypi_package("matplotlib", "https://pypi.org/simple") #Install matplotlib from given PyPI repository

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1598212531192_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**DATA LOAD**


In [17]:
#load stores
customers  = sqlContext.read.json('s3://tfmbigdata/Files_Segunda_Entrega/stores.json')

#load customers
customers  = sqlContext.read.json('s3://tfmbigdata/Files_Segunda_Entrega/Customers.json')

#load products
products  = sqlContext.read.json('s3://tfmbigdata/Files_Segunda_Entrega/Products.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#load tickets

text = sc.textFile("s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part0.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part1.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part2.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part3.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part4.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part5.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part6.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part7.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part8.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part9.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part10.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part11.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part12.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part13.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part14.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part15.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part16.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part17.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part18.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part19.json")

tickets = spark.read.json(text)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


**RFM**

In [3]:
tickets.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cardtype: string (nullable = true)
 |-- customerid: string (nullable = true)
 |-- datekey: string (nullable = true)
 |-- extendedamount: double (nullable = true)
 |-- linenumber: long (nullable = true)
 |-- originalamount: double (nullable = true)
 |-- productid: long (nullable = true)
 |-- quantity: double (nullable = true)
 |-- storeid: string (nullable = true)
 |-- ticketid: string (nullable = true)
 |-- totaldiscount: double (nullable = true)

In [3]:
#transform data types
tickets = tickets.withColumn("quantity", tickets["quantity"].cast(IntegerType()))
tickets = tickets.withColumn("unitprice", tickets["extendedamount"].cast(DoubleType()))
tickets = tickets.withColumn("date", to_date("datekey", "yyyy-mm-dd"))

#calculate recency 
max_df = tickets.select(max("date")).collect()
max_val = max_df[0][0]
tickets = tickets.withColumn('max_date',lit(max_val))
tickets = tickets.withColumn("recencydays", expr("datediff(max_date, date)"))

#calculate amount spent for item 
tickets = tickets.withColumn("total", round(tickets["unitprice"] * tickets["quantity"], 2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
#calculate recency frequency and money by customer

rfm_table = tickets.groupBy("customerid")\
                        .agg(min("recencyDays").alias("recency"), \
                             countDistinct("ticketid").alias("frequency"), \
                             sum("total").alias("monetary"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
rfm_table.write.format('csv').option('header',True).mode('overwrite').option('sep','|').save('/output.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**RFM RECONCILIATION - TO BE DELETED**

In [ ]:
test=tickets.groupBy("customerid").agg(countDistinct("ticketid").alias("count")).where('count > 1').sort('count', ascending=True).show()


In [ ]:
testing = tickets.select("customerid","ticketid","quantity","unitprice","date","max_date","recencydays"). \
filter((tickets.customerid  == "77000000157129940" )).orderBy("recencydays").show(100)

In [34]:
rfm_table.filter((rfm_table.customerid  == "77000000157129940" )).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

49729